In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis.align import rotation_matrix
import numpy as np
import glob as glob
import os as os

# A script to align a DNA base pair to the XY and place and translate that base pair to the origin.
# Written with help of Mateusz Bieniek

/Users/alex/anaconda3/lib/python3.6/site-packages/Bio/KDTree/__init__.py:27: BiopythonDeprecationWarning: Bio.KDTree has been deprecated, and we intend to remove it in a future release of Biopython. Please use Bio.PDB.kdtrees instead, which is functionally very similar.
  BiopythonDeprecationWarning,
/Users/alex/anaconda3/lib/python3.6/site-packages/MDAnalysis/due.py:88: UserWarning: No module named 'duecredit'
  warnings.warn(str(err))


In [2]:
def angle(p1, p2):
    def unit_vector(vector):
        return vector / np.linalg.norm(vector)
    v1_u = unit_vector(p1 - p2)
    v2_u = unit_vector([1, 0, 0])
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [3]:
def coord_axis(origin, x_direction, third_atom):
    def normalise(vector):
        return vector / np.linalg.norm(vector)
    x_dim = normalise(origin - x_direction)
    tmp_vec = normalise(origin - third_atom)
    # cross the vectors
    y_dim = normalise(np.cross(x_dim, tmp_vec))
    z_dim = normalise(np.cross(x_dim, y_dim))
    # coordination axis (x, y, z) vectors normalised
    return x_dim, y_dim, z_dim

In [4]:
# Change directories if necessary for different replicas
reac = '../../no_field/rep_101/optimised_struc/dna_r_opt.pdb'
ts1  = '../../no_field/rep_101/optimised_struc/ts1_opt.pdb'
prod = '../../no_field/rep_101/optimised_struc/dna_p_opt.pdb'

In [5]:
# Define the frame with the x y z
# Reference frame: x, y z
frame_ref = ((1, 0, 0), (0, 1, 0), (0, 0, 1))

In [6]:
# Align and orient the reac
u = mda.Universe(reac)

a1 = u.select_atoms('bynum 83')
a2 = u.select_atoms('bynum 682')

# check the angle to the X-axis in the beginning
a = angle(a2[0].position, a1[0].position)
print("Original angle", np.rad2deg(a))

# pick atoms which will be superimposed with your reference frame
# argument: 1) origin, 2) x axis (y and z should be 0), 3) where y should be 0
ourframe = u.select_atoms('bynum 681', 'bynum 83', 'bynum 85').positions
normalised = coord_axis(*ourframe)

# find a rotation for the system so that the selected atoms perfectly overlap with the reference frame
R, rmsd = rotation_matrix(normalised, frame_ref)

# apply the rotation to the entire system
u.atoms.rotate(R)

# check if the angle to the x-axis is smaller
a = angle(a2[0].position, a1[0].position)
print("Corrected angle", np.rad2deg(a))

# Move index 681 [H41] to origin 
u.atoms.positions[681]
# Translate coordinates by position of H41
translated = u.atoms.positions - u.atoms.positions[681]
# Update the position
u.atoms.positions = translated

# Save pdb of DNA with H41 centred on origin and GC base pair (resid 3 and 22) aligned to the XZ axis
u.atoms.write('./rep_101/a_reac/reac_o.pdb')

Original angle 12.560147157351029
Corrected angle 2.163922467635587


In [7]:
# Align and orient the ts1
u = mda.Universe(ts1)

a1 = u.select_atoms('bynum 83')
a2 = u.select_atoms('bynum 682')

# check the angle to the X-axis in the beginning
a = angle(a2[0].position, a1[0].position)
print("Original angle", np.rad2deg(a))

# pick atoms which will be superimposed with your reference frame
# argument: 1) origin, 2) x axis (y and z should be 0), 3) where y should be 0
ourframe = u.select_atoms('bynum 681', 'bynum 83', 'bynum 85').positions
normalised = coord_axis(*ourframe)

# find a rotation for the system so that the selected atoms perfectly overlap with the reference frame
R, rmsd = rotation_matrix(normalised, frame_ref)

# apply the rotation to the entire system
u.atoms.rotate(R)

# check if the angle to the x-axis is smaller
a = angle(a2[0].position, a1[0].position)
print("Corrected angle", np.rad2deg(a))

# Move index 681 [H41] to origin 
u.atoms.positions[681]
# Translate coordinates by position of H41
translated = u.atoms.positions - u.atoms.positions[681]
# Update the position
u.atoms.positions = translated

# Save pdb of DNA with H41 centred on origin and GC base pair (resid 3 and 22) aligned to the XZ axis
u.atoms.write('./rep_101/b_ts1/ts1_o.pdb')

Original angle 12.527975912461418
Corrected angle 2.8140383292902262


In [8]:
# Align and orient the product
u = mda.Universe(prod)

a1 = u.select_atoms('bynum 83')
a2 = u.select_atoms('bynum 682')

# check the angle to the X-axis in the beginning
a = angle(a2[0].position, a1[0].position)
print("Original angle", np.rad2deg(a))

# pick atoms which will be superimposed with your reference frame
# argument: 1) origin, 2) x axis (y and z should be 0), 3) where y should be 0
ourframe = u.select_atoms('bynum 681', 'bynum 83', 'bynum 85').positions
normalised = coord_axis(*ourframe)

# find a rotation for the system so that the selected atoms perfectly overlap with the reference frame
R, rmsd = rotation_matrix(normalised, frame_ref)

# apply the rotation to the entire system
u.atoms.rotate(R)

# check if the angle to the x-axis is smaller
a = angle(a2[0].position, a1[0].position)
print("Corrected angle", np.rad2deg(a))

# Move index 681 [H41] to origin 
u.atoms.positions[681]
# Translate coordinates by position of H41
translated = u.atoms.positions - u.atoms.positions[681]
# Update the position
u.atoms.positions = translated

# Save pdb of DNA with H41 centred on origin and GC base pair (resid 3 and 22) aligned to the XZ axis
u.atoms.write('./rep_101/e_product/prod_o.pdb')

Original angle 12.30262310731649
Corrected angle 2.9851775733401484
